In [1]:
#Importing Library and setting environment path
import os
import sys

In [2]:
os.environ['SPARK_HOME'] = "C:/Users/Abhinav/Downloads/spark-2.0.2-bin-hadoop2.7"
sys.path.append("C:/Users/Abhinav/Downloads/spark-2.0.2-bin-hadoop2.7/bin")
sys.path.append("C:/Users/Abhinav/Downloads/spark-2.0.2-bin-hadoop2.7/python")
sys.path.append("C:/Users/Abhinav/Downloads/spark-2.0.2-bin-hadoop2.7/python/pyspark")
sys.path.append("C:/Users/Abhinav/Downloads/spark-2.0.2-bin-hadoop2.7/python/lib")
sys.path.append("C:/Users/Abhinav/Downloads/spark-2.0.2-bin-hadoop2.7/python/lib/pyspark.zip")
sys.path.append("C:/Users/Abhinav/Downloads/spark-2.0.2-bin-hadoop2.7/python/lib/py4j-0.10.3-src.zip")
sys.path.append("C:/Program Files/Java/jre1.8.0_77/bin")

In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
#website = "nytimes"

In [5]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

data = sc.textFile("C:/Users/Abhinav/Google Drive/Big Data Project/Datasets/All combined/all_data.txt")

Tokenizer

In [6]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

Removing punctuations and numbers

In [7]:
import re
def removePunctuation(text):
    inter1 = re.sub('[^a-z| |]', '', text.strip().lower().encode('utf-8'))
    inter2 = inter1.replace("\'", "")
    return inter2

In [8]:
data_sanspunct = data.map(removePunctuation)

Removing Stop Words

In [9]:
from stop_words import get_stop_words
en_stop = get_stop_words('en')

def removeStopWords(line):
    tokens = tokenizer.tokenize(line)
    words = [i for i in tokens if not i in en_stop]
    return words

In [10]:
data_sansstopwords = data_sanspunct.map(removeStopWords)

Stemming

In [11]:
from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()

def stemmingWords(words):
    stemmed_word = [p_stemmer.stem(i) for i in words]
    return stemmed_word

In [12]:
data_sansstemmingwords = data_sansstopwords.map(stemmingWords)

Creating document-term matrix

In [13]:
import gensim 
from gensim import corpora, models

def documentTermMatrix(texts):
    dictionary = corpora.Dictionary(texts)
    return dictionary

C:\Users\Abhinav\AppData\Local\Enthought\Canopy\User\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Abhinav\AppData\Local\Enthought\Canopy\User\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [14]:
corpus = data_sansstemmingwords.map(lambda word:documentTermMatrix(word))

In [15]:
word_list = data_sansstemmingwords.collect()[0]

In [16]:
doc_clean = [doc.split() for doc in word_list]
dictionary = corpora.Dictionary(doc_clean)
corpus = [dictionary.doc2bow(text) for text in doc_clean]

In [17]:
import gensim
from gensim import corpora, models

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word = dictionary, iterations=3, random_state=7)

In [18]:
# select top 50 words for each of the 20 LDA topics
top_words = [[str(word[0].encode('utf-8')) for word in ldamodel.show_topic(topicno, topn=50)] for topicno in range(ldamodel.num_topics)]

In [19]:
import itertools
import  numpy as np
# get all top 50 words in all 20 topics, as one large set
all_words = set(itertools.chain.from_iterable(top_words))

# for each topic, replace a word at a different index, to make it more interesting
#replace_index = np.random.randint(0, 10, ldamodel.num_topics)

replacements = []
for topicno, words in enumerate(top_words):
#    other_words = all_words.difference(words)
#    replacement = np.random.choice(list(other_words))
#    replacements.append((words[replace_index[topicno]], replacement))
#    words[replace_index[topicno]] = replacement
    print("Topic %i: %s" % (topicno, ' '.join(words[:10])))

Topic 0: time know happen wasnut uthi stand let ever put noth
Topic 1: just come two race still better ufollow ulanda includ expert
Topic 2: player guy complet give comment field went victori michigan uniform
Topic 3: right use chang doesnut late overal america anyth car usual
Topic 4: get countri presid report second bring th highli either line
Topic 5: u leagu slalom pick real thereu stop packer obvious bryant
Topic 6: said game uth fan shot era life news uthatu winud
Topic 7: ui start next made keep mani candid left bradi note
Topic 8: year landau return believ offici four away uwhat washington although
Topic 9: need take lot ball deal papadaki uhe live spent less
Topic 10: like got came new injuri show issu defens roster uin
Topic 11: one donut goal call coach young help eight wear wild
Topic 12: man norman pat train turn cut sure goodel respect thought
Topic 13: way well won miss twitter fact test place dak russia
Topic 14: none never szew told home prix coupl due terribl uuch
Top

In [22]:
top_words

[['time',
  'know',
  'happen',
  'wasnut',
  'uthi',
  'stand',
  'let',
  'ever',
  'put',
  'noth',
  'gonna',
  'actual',
  'third',
  'care',
  'yard',
  'target',
  'everyth',
  'elector',
  'obama',
  'heud',
  'bowl',
  'biggest',
  'guess',
  'reach',
  'cook',
  'knee',
  'expand',
  'father',
  'prove',
  'choos',
  'correct',
  'intern',
  'comeback',
  'wheel',
  'began',
  'educ',
  'poor',
  'panther',
  'senat',
  'youuv',
  'grew',
  'arduou',
  'nhk',
  'met',
  'prepar',
  'truth',
  'shut',
  'pile',
  'citizen',
  'intent'],
 ['just',
  'come',
  'two',
  'race',
  'still',
  'better',
  'ufollow',
  'ulanda',
  'includ',
  'expert',
  'hard',
  'drop',
  'minor',
  'professor',
  'troll',
  'nascar',
  'realiti',
  'content',
  'answer',
  'ufriday',
  'conspiraci',
  'ucsup',
  'uclittl',
  'acknowledgedxalanda',
  'everybodi',
  'moment',
  'son',
  'definit',
  'notic',
  'school',
  'neg',
  'symptom',
  'youngest',
  'star',
  'iull',
  'hater',
  'multipli',

In [23]:
import pandas as pd
website_list = ["animalplanet","natgeo","Discovery","TMZ","HISTORY","NBCSports","ESPN","nytimes","cnn","bbc","theatlantic"]
final = pd.DataFrame({"Topics":np.arange(30)})
for website in website_list:
    text = sc.textFile("C:/Users/Abhinav/Google Drive/Big Data Project/Datasets/Website Scraped/"+website+"_top10_posts_BOW_*")
    text = text.map(removePunctuation)
    text = text.map(removeStopWords)
    text = text.map(stemmingWords)

    word_list_1 = text.collect()[0]

    #doc_clean = [doc.split() for doc in word_list]
    #dictionary = corpora.Dictionary(doc_clean)
    #corpus = [dictionary.doc2bow(text) for text in doc_clean]

    doc_clean_1 = [doc.split() for doc in word_list_1]
    #dictionary_1 = corpora.Dictionary(doc_clean_1)
    corpus_1 = [dictionary.doc2bow(text) for text in doc_clean_1]


    #doc_clean_1 = [doc.split() for doc in word_list_1]
    vec_bow = dictionary.doc2bow(word_list_1)
    vec_lsi = ldamodel[vec_bow] # convert the query to LSI space
    vec_lsi

    output = pd.DataFrame(vec_lsi)
    output.columns = ["Topics", website]

    final = final.merge(output)
    final.to_csv("topic_probability_clustering.csv",index=False)

In [24]:
from numpy import array
from math import sqrt

from pyspark.mllib.clustering import KMeans, KMeansModel

In [34]:
# Load and parse the data
data = sc.textFile("C:/Users/Abhinav/Google Drive/Big Data Project/Codes/clustering_probability.csv")
parsedData = data.map(lambda line: array([float(x) for x in line.split(',')]))
parsedData.take(5)

[array([ 0.026723  ,  0.02823678,  0.03274008,  0.0498035 ,  0.03433012,
         0.05815009,  0.03493342,  0.03341556,  0.02631256,  0.03590577,
         0.03022529,  0.0221985 ,  0.01878709,  0.01007155,  0.0252264 ,
         0.02684838,  0.02098522,  0.0371751 ]),
 array([ 0.04294931,  0.03910734,  0.02518367,  0.02439072,  0.02551375,
         0.03770163,  0.03053104,  0.03304557,  0.03098319,  0.02442024,
         0.03012939,  0.02747059,  0.03280636,  0.03814229,  0.05446749,
         0.03616599,  0.0244186 ,  0.02594528]),
 array([ 0.01909358,  0.01454822,  0.03007739,  0.01433485,  0.02587966,
         0.01481093,  0.05056253,  0.02212454,  0.02091468,  0.01557134,
         0.0206006 ,  0.02446442,  0.03295419,  0.03745415,  0.02287935,
         0.03567992,  0.01560564,  0.04768854]),
 array([ 0.02255615,  0.04329312,  0.01142147,  0.02494486,  0.03707364,
         0.01840014,  0.03900605,  0.03236733,  0.03451242,  0.05260936,
         0.02295111,  0.04821898,  0.02060314,  0.

In [39]:
parsedData.collect()

[array([ 0.026723  ,  0.02823678,  0.03274008,  0.0498035 ,  0.03433012,
         0.05815009,  0.03493342,  0.03341556,  0.02631256,  0.03590577,
         0.03022529,  0.0221985 ,  0.01878709,  0.01007155,  0.0252264 ,
         0.02684838,  0.02098522,  0.0371751 ]),
 array([ 0.04294931,  0.03910734,  0.02518367,  0.02439072,  0.02551375,
         0.03770163,  0.03053104,  0.03304557,  0.03098319,  0.02442024,
         0.03012939,  0.02747059,  0.03280636,  0.03814229,  0.05446749,
         0.03616599,  0.0244186 ,  0.02594528]),
 array([ 0.01909358,  0.01454822,  0.03007739,  0.01433485,  0.02587966,
         0.01481093,  0.05056253,  0.02212454,  0.02091468,  0.01557134,
         0.0206006 ,  0.02446442,  0.03295419,  0.03745415,  0.02287935,
         0.03567992,  0.01560564,  0.04768854]),
 array([ 0.02255615,  0.04329312,  0.01142147,  0.02494486,  0.03707364,
         0.01840014,  0.03900605,  0.03236733,  0.03451242,  0.05260936,
         0.02295111,  0.04821898,  0.02060314,  0.

In [26]:
# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 7, maxIterations=10,
                        runs=10, initializationMode="random", seed = 1234)

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

# Save and load model
#clusters.save(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")
#sameModel = KMeansModel.load(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")

C:/Users/Abhinav/Downloads/spark-2.0.2-bin-hadoop2.7/python\pyspark\mllib\clustering.py:347: UserWarning: The param `runs` has no effect since Spark 2.0.0.
  warnings.warn("The param `runs` has no effect since Spark 2.0.0.")


Within Set Sum of Squared Error = 0.153241988881


In [44]:
for i in range(0,8):
    list_i = list()
    for j in range(0,5):
        list_i.append(j)
    print list_i

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


In [38]:
np.sqrt(sum(np.square(parsedData.collect()[1]-clusters.clusterCenters[1])))

0.14891262255219834

In [47]:
for i in range(0,7):
    list_i = list()
    for j in range(0,11):
        list_i.append(np.sqrt(sum(np.square(parsedData.collect()[j]-clusters.clusterCenters[i]))))
    print list_i

[0.062655952452143088, 0.035623685735485966, 0.068981238901420008, 0.060558839847077252, 0.057060871363185488, 0.030302515466788991, 0.0, 0.054868605109333106, 0.13435648465484759, 0.13408264994666647, 0.043626709584165521]
[0.15627587002438359, 0.14891262255219834, 0.16711167291195816, 0.15301961130377109, 0.15197184288774399, 0.13748400751901571, 0.13421942278980323, 0.15450721388196956, 0.00023987233344843772, 0.00023987233344844477, 0.14233062643050789]
[0.062655952452143088, 0.035623685735485966, 0.068981238901420008, 0.060558839847077252, 0.057060871363185488, 0.030302515466788991, 0.0, 0.054868605109333106, 0.13435648465484759, 0.13408264994666647, 0.043626709584165521]
[0.064449714428961372, 0.020551255414960209, 0.05921210545593715, 0.066106101970974246, 0.043934878850099675, 0.022489341818718103, 0.033604276711877976, 0.035259596871661157, 0.14384214983383559, 0.14360058576894991, 0.020317224843583281]
[0.075763559771385408, 0.067225776757201886, 0.069659943355994236, 0.0, 0.

IndexError: list index out of range

In [30]:
clusters.centers

[array([ 0.03222045,  0.04118483,  0.02502697,  0.0470347 ,  0.02190892,
         0.02268673,  0.0199363 ,  0.03487252,  0.02681475,  0.02988978,
         0.02413561,  0.03863355,  0.03119778,  0.03880149,  0.04705844,
         0.0357354 ,  0.02817384,  0.02761497]),
 array([ 0.03524044,  0.01072243,  0.02810085,  0.06828123,  0.0217448 ,
         0.02571673,  0.01101781,  0.04704815,  0.05100597,  0.01771262,
         0.01458848,  0.10316807,  0.01858107,  0.0238668 ,  0.03982263,
         0.07477872,  0.12384077,  0.01089842]),
 array([ 0.03222045,  0.04118483,  0.02502697,  0.0470347 ,  0.02190892,
         0.02268673,  0.0199363 ,  0.03487252,  0.02681475,  0.02988978,
         0.02413561,  0.03863355,  0.03119778,  0.03880149,  0.04705844,
         0.0357354 ,  0.02817384,  0.02761497]),
 array([ 0.03607311,  0.03526311,  0.03070915,  0.02538596,  0.0242442 ,
         0.034811  ,  0.03220426,  0.03001514,  0.02681908,  0.0306746 ,
         0.02906785,  0.03109322,  0.03615635,  0.